In [1]:
import cv2
import os
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot as plt
from matplotlib.path import Path
from scipy import ndimage
from skimage import measure, color, io, filters,feature, morphology, segmentation, util
from scipy import ndimage as ndi
from scipy.stats.kde import gaussian_kde
from scipy.ndimage.filters import gaussian_filter
import pandas as pd
import seaborn as sns
import csv
import glob
import warnings

try:
    import numexpr
except ImportError:
    numexpr = None

from skimage.transform import integral_image


In [411]:
def fill_holes(img):
    im_th=img
    im_floodfill = im_th.copy()

    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);

    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    return im_out

##__init__path
path=''#Path of Fiji proccessed Images
files = glob.glob(path+'/**/*.tif', recursive=True)
##__init__constants
MaxArea=2500
MinArea=200
fac= 10
df=pd.DataFrame()
fmax=0
wt=[]


##__main__loop
##Image initializing
for i in range(0,len(files)):
    file = files[i]
    donor=os.path.splitext(os.path.basename(file))[0]
    I0 =cv2.imread(file)
    cells = I0[:,:,0]
    ## GrayscaleThresholding
    ret1, thresh = cv2.threshold(cells, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #thresh=phansalkar(cells)
    ##morphological binary improvement
    fill = fill_holes(thresh)
    fill = np.uint8(fill)

    kernel= np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(fill, cv2.MORPH_OPEN, kernel, iterations = 2)
    ## background
    sure_bg = cv2.dilate(opening,kernel,iterations=2)
    ##distance_tranformation to find peaks
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2,5)
    ##Forground
    ret2, sure_fg = cv2.threshold(dist_transform,0.25*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)

    ##difference For - background
    unknown = cv2.subtract(sure_bg,sure_fg)

    ##ROI-Labeling
    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0
    #plt.imshow(markers, cmap='jet')  

    ##markerbased Watershed
    markers= cv2.watershed(I0,markers)
    vars()[donor+'_marker']=markers

    ##getting Region properties
    regions = measure.regionprops_table(markers, intensity_image=cells, properties=('label','centroid','area'))
    cents= pd.DataFrame(regions)
    cents=cents[(cents["area"]<MaxArea) & (cents["area"]>MinArea)]

    x=cents["centroid-0"]
    y=cents["centroid-1"]

    #Grid for Heatmnap
    
    xx, yy = np.mgrid[0:cells.shape[0]:complex(0,int(cells.shape[0]/fac)), 0:cells.shape[1]:complex(0,int(cells.shape[1]/fac))]
    #]
    position = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x,y])
    
    #gaussian KDE with fixed bandwith
    vars()[donor+"kernel"] = gaussian_kde(values,bw_method=0.2)
    
    vars()[donor]= np.reshape(vars()[donor+"kernel"](position).T,xx.shape)
    
    #Donorlist for variables to call depending on the filename   
    wt.append(donor)

In [296]:
clrnorm=[]

# cropping the Heatmap depending on the Lobule shape
for i in range(0,len(wt)):
    fig,ax = plt.subplots(figsize=(10,10))
    df_coord=pd.read_csv(path+'/'+wt[i]+'_coords.csv',delimiter=',',header=0,decimal=',')
    coords=np.array(df_coord.astype(int))
    yscal=cells.shape[1]/vars()[wt[i]].shape[1]
    xscal=cells.shape[0]/vars()[wt[i]].shape[0]
    
    pc=[]
    pc = coords[1:].tolist()
    pc=np.divide(pc,[xscal,yscal])
    #pc = np.vstack((pc)).T
    
    nr,nc = vars()[wt[i]].shape
    ygrid, xgrid = np.mgrid[:nr, :nc]
    xypix = np.vstack((xgrid.ravel(), ygrid.ravel())).T
    
    pth = Path(pc, closed=False)
    mask = pth.contains_points(xypix)
    mask = mask.reshape(vars()[wt[i]].shape)
    vars()[wt[i]+'masked'] =vars()[wt[i]]*mask
    
    #min and max heatmap values for normalization of Heatmap plot
    clrnorm.append(vars()[wt[i]+'masked'].max())
    clrnorm.append(vars()[wt[i]+'masked'][np.nonzero(vars()[wt[i]+'masked'])].min())
    
    plt.imshow(vars()[wt[i]+'masked'])
    plt.title(wt[i],fontsize=20)
    plt.savefig(path+'/LobulesVis/mask/'+wt[i]+'_KDE_LobuleVis_Mskd.png')
    plt.close(fig)

In [358]:
cmap=plt.get_cmap('viridis')
norm= plt.Normalize(min(clrnorm),max(clrnorm))
()


for i in range(0,len(wt)):
    fig,ax = plt.subplots(figsize=(10,10))
    df_coord=pd.read_csv(path+'/'+wt[i]+'_coords.csv',delimiter=',',header=0,decimal=',')
    coords=np.array(df_coord.astype(int))
    yscal=cells.shape[1]/vars()[wt[i]].shape[1]
    xscal=cells.shape[0]/vars()[wt[i]].shape[0]
    poly=[]
    poly = coords[1:].tolist()
    poly.append(poly[0])
    poly=np.divide(poly,[xscal,yscal])
    px,py = zip(*poly)
    
    plt.imshow(vars()[wt[i]+'masked'] ,norm=norm,cmap=cmap)
    #plt.title(wt[i],fontsize=30)

    
    
    clr=iter(plt.cm.rainbow(np.linspace(0,1,len(coords)-1)))
    rows = len(coords)-1
   
    for j in range(0,rows):
        c=next(clr)
        
        plt.plot([coords[0][0]/xscal,coords[j+1][0]/xscal], [coords[0][1]/yscal,coords[j+1][1]/yscal],c=c )
        plt.plot(px,py, c='r')
    
    plt.savefig(path+'/LobulesVis/RoiMaps/'+wt[i]+'_KDE_LobuleVisMSKD.png')
    plt.close(fig)
    
    num = 101
    ziarr =np.array([]).reshape(0,num)

    #mapping portal central axis on heatmap and getting density along axis

    for k in range(1,len(coords)):
        lx0 = coords[0][0]/xscal
        ly0 = coords[0][1]/yscal
        # These are in _pixel_ coordinates!!
        lx1 = coords[k][0]/xscal
        ly1 = coords[k][1]/yscal
        #lx1, ly1 = coords[i]


        lx, ly = np.linspace(lx0, lx1, num), np.linspace(ly0, ly1, num)
        zi=ndi.map_coordinates(vars()[wt[i]], np.vstack((lx,ly)),mode='nearest')
        ziarr=np.vstack((ziarr,zi))

    #DataFrame of portal central axis per donor and mean and standar diviation    
    vars()[wt[i]+'df']= pd.DataFrame(ziarr.T)
    vars()[wt[i]+'df']["Mean"]=vars()[wt[i]+'df'].iloc[:].mean(axis=1)
    vars()[wt[i]+'df']["std"] = vars()[wt[i]+'df'].iloc[:,:-1].std(axis=1)
    vars()[wt[i]+'df']["Donor"]=wt[i]

In [383]:
df_don=pd.DataFrame()
donarr=[[0,9],[10,21],[22,36]] #position numbers of tissue samples for ranging

for i in range(0,len(donarr)): 
    Don=wt[donarr[i][0]][:-1]


    for i in range(donarr[i][0],donarr[i][1]+1):
        df_T=vars()[wt[i]+'df'].iloc[:,:-3].T
        df_don=pd.concat([df_don,df_T],axis=0)

    df_don=df_don.T
    df_don["Mean"]=df_don.iloc[:].mean(axis=1)
    df_don["std"] = df_don.iloc[:,:-1].std(axis=1)

    #df_don.to_excel(os.path.dirname(file)+'/'+donor+'_Means.xlsx')

    fig,ax1 = plt.subplots(figsize=(10,10))

    x= np.linspace(0, len(df_don),len(df_don))
    #ax1.set_title('MeanofDonor '+Don, fontsize=30)
    ax1.plot(df_don["Mean"],label='mean', color='blue')
    ax1.fill_between(x,df_don["Mean"]-df_don["std"],df_don["Mean"]+df_don["std"],alpha=0.5,label='error', color='lightblue')
    ax1.legend()
    ax1.set_ylim(([0.7*1e-8,1.2*1e-8]))
    fig.savefig(path+'/LobulesVis/MoD/MoCtrl_02.png')
    plt.close(fig)

In [392]:
#plotting all means per Lobule

for i in range(0,len(wt)):
    fig,ax1 = plt.subplots(figsize=(10,10))
    df_data=vars()[wt[i]+'df']
    x= np.linspace(0, len(df_data),len(df_data))
    #ax1.set_title('MeanofLobule '+wt[i], fontsize=30)
    ax1.plot(df_data["Mean"],label='mean', color='blue')
    ax1.fill_between(x,df_data["Mean"]-df_data["std"],df_data["Mean"]+df_data["std"],alpha=0.5,label='error', color='lightblue')
    ax1.legend()
    ax1.set_ylim(([0.6*1e-8,1.4*1e-8]))
    fig.savefig(path+'/LobulesVis/MoL/'+wt[i]+'_MoL.png')
    plt.close(fig)